In [1]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

## Primer Cuatrimestre de 2017. Examen parcial, primera oportunidad.

Se tiene información estadística de la temporada regular de todos los
jugadores de la NBA en un RDD de tuplas con el siguiente formato:
(id_jugador, nombre, promedio_puntos, promedio_asistencias, promedio_robos,
promedio_bloqueos, promedio_rebotes, promedio_faltas). 

Un analista de la
cadena ESPN está trabajando con un RDD que corresponde a la primera ronda
de playoffs y que tiene el siguiente formato: (id_jugador, id_partido, timestamp,
cantidad_puntos, cantidad_rebotes, cantidad_bloqueos, cantidad_robos,
cantidad_asistencias, cantidad_faltas). 

<b>En base a estos RDDs se quiere
programar en PySpark un programa que genere un RDD con los nombres (sin
duplicados) de los jugadores que lograron en algún partido de playoffs una
cantidad de asistencias mayor a su promedio histórico.</b> (****) (15 pts)

In [22]:
# usamos para simplificar el formato, que puede obtenerse con un map.
# (id_jugador, nombre, promedio_asistencias)
players_all_time_stats = [
    (1, 'Manu Ginobili', 800),
    (2, 'Kobe Bryant', 100),
    (3, 'Marc Gasol', 25),
    (4, 'James Harden', 1000)]

# usamos para simplificar el formato, que puede obtenerse con un map.
# (id_jugador, id_partido, timestamp, cantidad_asistencias)
scores = [
  (1, 1, 1, 100),
  (1, 1, 3, 100),
  (2, 1, 1, 150),
  (2, 1, 3, 150),
  (3, 2, 2, 50),
  (3, 2, 3, 50),      
  (1, 2, 1, 150),
  (1, 2, 3, 150),
]

players_all_time_stats_rdd = sc.parallelize(players_all_time_stats)
scores_rdd = sc.parallelize(scores)

In [35]:
stats_season = rdd_scores.map(lambda x: ((x[0],x[1]),x[3])).map(lambda a: (a[0][0], a[1])).distinct()
stats_season.collect()

[(1, 150), (2, 150), (3, 50), (1, 100)]

In [36]:
info_players = players_all_time_stats_rdd.map(lambda x: (x[0],(x[1],x[2])))
info_players.collect()

[(1, ('Manu Ginobili', 800)),
 (2, ('Kobe Bryant', 100)),
 (3, ('Marc Gasol', 25)),
 (4, ('James Harden', 1000))]

In [37]:
stats = stats_season.join(info_players)
stats.collect()

[(1, (150, ('Manu Ginobili', 800))),
 (1, (100, ('Manu Ginobili', 800))),
 (2, (150, ('Kobe Bryant', 100))),
 (3, (50, ('Marc Gasol', 25)))]

In [41]:
stats.filter(lambda x: x[1][0] > x[1][1][1]).map(lambda x: x[1][1][0]).collect()

['Kobe Bryant', 'Marc Gasol']

## Parcial 2016 2do Cuatrimestre. Examen parcial, segunda oportunidad.

En este ejercicio queremos programar un sistema que recomiende
textos a usuarios en base a sus gustos sobre ciertos términos (palabras).

Se cuenta con un RDD de textos de la forma (docId, texto) donde texto
es un string de longitud variable. 

Además contamos con un RDD que
indica qué términos le gustan o no a cada usuario de la forma (userId,
término, score) por ejemplo (23, “calesita”, -2). 

Se pide programar en Spark un programa que calcule el score total de cada documento para cada usuario generando un RDD de la forma (userId, docId, score) en donde el score es simplemente la suma de los scores del usuario para
los términos que aparecen en el documento. 

Puede haber términos en los documentos para los cuales no exista score de algunos usuarios, en
estos casos simplemente los consideramos neutros (score=0)

In [47]:
documents = [
    (1, 'pablo honey'),
    (2, 'the bends'),
    (3, 'ok computer'),
    (4, 'kid a'),
    (5, 'amnesiac'),
    (6, 'hail to the thief'),
    (7, 'in rainbows'),
    (8, 'the king of limbs'),
    (9, 'a moon shaped pool')
]

scores = [
    ('thom', 'pablo', 1),
    ('thom', 'honey', 1),
    ('martin', 'pablo', -1),
    ('martin', 'honey', -1),
    ('martin', 'ok', 30),
    ('martin', 'computer', 30)
]

In [48]:
rdd_documents = sc.parallelize(documents,4)
rdd_scores = sc.parallelize(scores,4)

In [51]:
words_by_documents_rdd = rdd_documents.flatMap(lambda x: [(word,x[0]) for word in x[1].split()])

In [52]:
scores_for_join_rdd = rdd_scores.map(lambda x: (x[1],(x[0],x[2])))

In [58]:
# ('pablo', (1,('thom', 1))
words_by_documents_rdd.join(scores_for_join_rdd).map(lambda x: ((x[1][1][0], x[1][0]), x[1][1][1]))\
    .reduceByKey(lambda x,y: x+y).map(lambda x: (x[0][0],x[0][1],x[1])).collect()

[('thom', 1, 2), ('martin', 1, -2), ('martin', 3, 60)]

## 2015-02 2do Recuperatorio
Un telescopio registra automaticamente la luminosidad de distintas
estrellas generando un RDD con registros de tipo (star_id,
magnitude,spectral_type, timestamp). Queremos obtener un listado de
estrellas que tienen el mismo tipo espectral e igual promedio de
magnitud una vez redondeado el mismo a un decimal. El resultado
debe ser una lista en donde cada elemento de la lista es una lista de ids
de estrellas similares.

In [59]:
stars = [
    (1, 5, 1, 1),
    (2, 10, 1, 1),
    (3, 6, 1, 1),
    (4, 5.5, 2, 1),
    (1, 6, 1, 2),
    (2, 9, 1, 2),
    (3, 5, 1, 2),
    (1, 5.5, 1, 3),
    (2, 11, 1, 3),
    (3, 5.5, 1, 3)
]
rdd = sc.parallelize(stars)

In [76]:
rdd.map(lambda x: ((x[0], x[2]), (x[1], 1)))\
    .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))\
    .map(lambda x: ((x[0][1], round(x[1][0]/x[1][1],1)), x[0][0]))\
    .groupByKey()\
    .map(lambda x: (x[0], list(x[1])))\
    .collect()

[((1, 5.5), [1, 3]), ((1, 10.0), [2]), ((2, 5.5), [4])]

## Organización de Datos 75.06. Segundo Cuatrimestre de 2015. Examen parcial, segunda oportunidad:
1) Se tiene un RDD con libros en donde cada registro es un texto. Se
pide obtener todos los anagramas de mas de 7 letras que puedan
encontrarse. El formato de salida debe ser una lista de listas en donde
cada lista tiene un conjunto de palabras que son anagramas. Ejemplo:
[[discounter,introduces,reductions],[percussion,supersonic]...]
 (4*) (15 pts)

In [16]:
texts = ['the discounter reductions',
        'the air introduces supersonic sounds',
        'anana no es lo misma que naana',
        'percussion and fourious introduces panic']

rdd = sc.parallelize(texts)
rdd.flatMap(lambda x: x.split()).filter(lambda x: len(x)>=7).distinct()\
.map(lambda x: (''.join(sorted(x)),x)).groupByKey().map(lambda x: list(x[1])).filter(lambda x: len(x)>1).collect()

[['percussion', 'supersonic'], ['discounter', 'reductions', 'introduces']]